<h1 align="center"><b>AI Lab: Computer Vision and NLP</b></h1>
<h3 align="center">Lesson 19: Convolutional Neural Networks</h3>

---

**C**onvolutional **N**eural **N**etworks (**CNN**s) are a specific type of neural networks that are composed by convolution layers and pooling layers. At the end of the CNN we still have a MLP. The purpose of the CNN is to extract features from the images (in fact, CNNs are usually used with images).